As starting point we take a SpatialData `.zarr` file, and preprocess it, so it can be visualized by Vitessce.

A SpatialData `.zarr` file can be created e.g. as follows:

```
import harpy as hp
sdata=hp.datasets.xenium_example( output=".../sdata_xenium.zarr" ) # change this output path
```

In [ ]:
import os

from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
    MultiImageWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

from vitessce.data_utils import multiplex_img_to_ome_zarr

In [ ]:
from spatialdata import read_zarr

path = "/Users/arnedf/VIB/DATA/test_data" # change this path

sdata = read_zarr(  os.path.join( path, "sdata_xenium.zarr")  )

In [ ]:
sdata[ "table_global" ]

Create `.zarr` files for `Vitessce`.

In [ ]:
output_path_img= os.path.join( path,  "raw_image.ome.zarr" )
output_path_masks=os.path.join( path, "masks.ome.zarr" )
output_path_adata=os.path.join( path, "adata.zarr" )

img_layer = "morphology_focus_global"
labels_layer = "cell_labels_global"
table_layer = "table_global"

array=sdata[ img_layer ][ "scale0" ]["image"].data.compute()
multiplex_img_to_ome_zarr( img_arr=array, channel_names=[ "grey" ], axes="cyx",  output_path=output_path_img)
array=sdata[ labels_layer ]["scale0"][ "image" ].data.compute()
multiplex_img_to_ome_zarr( img_arr=array[ None,... ], channel_names=[ "grey" ], axes="cyx",  output_path=output_path_masks )

adata=sdata[ table_layer ]

adata.obs.index = adata.obs[ "cell_ID" ]
adata.obs.index.name = None

adata = optimize_adata(
    adata,
    obs_cols=["total_counts"],
    obsm_keys=[ "spatial"],
    optimize_X=True,
    to_dense_X=True,
)
adata.write_zarr(output_path_adata, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

Now copy these files to an S3 bucket using Globus.

Lets check locally.

# 

In [ ]:
from vitessce import OmeZarrWrapper

vc = VitessceConfig(schema_version="1.0.15", name='xenium data', description='xenium data')

dataset = vc.add_dataset(name='Xenium').add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeZarrWrapper(img_path=output_path_masks, is_bitmask=True, name='segmentations'),
            OmeZarrWrapper(img_path=output_path_img, is_bitmask=False, name='image'),
        ],
 )
)

dataset.add_object(AnnDataWrapper(
        adata_path=output_path_adata,
        obs_locations_path="obsm/spatial",
        obs_set_paths=["obs/total_counts"],
        obs_set_names=["Total counts"],
        obs_feature_matrix_path="X",
        coordination_values={
          "obsType": "cell",
          "featureType": "gene",
          "featureValueType": "expression"
        }

    ),
)

In [18]:
spatial_plot = vc.add_view(cm.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)

In [19]:
vc.layout((spatial_plot | layer_controller/ genes));

In [ ]:
from IPython.display import display, HTML
url = vc.web_app()
display(HTML(f'<a href="{url}" target="_blank">Open in Vitessce</a>'))

You can create the `Vitessce` config, and copy the files using the following code, but it is probably more efficient to copy the files using S3, and create and manipulate the config manually.

In [ ]:
import boto3
import json

s3 = boto3.resource(
    service_name='s3',
    endpoint_url="https://objectstor.vib.be",
    aws_access_key_id=os.environ['ACCESS_KEY'],
    aws_secret_access_key=os.environ['SECRET_KEY'],
)

config_dict = vc.export(to='S3', s3=s3, bucket_name="spatial-hackathon-public", prefix='_test_vitessce_1')

with open( "config_xenium.json", "w" ) as f: # this .json should be uploaded to the bucket, note that file paths still need to be updated
    json.dump(config_dict, f, indent=4)